In [1]:
#--------------------------------------------------------------------
# Talha Shahzad
# 21i-0540
# Section : C
# Final Project
# Artificial Intelligence
#--------------------------------------------------------------------

In [2]:
import random

In [3]:
#Initiallization 
Days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
TotalClassesPerDay = 6

Courses = ["Programming Fundamentals", "Data Structures", "Algorithms", "Database Management Systems", "Operating Systems", "Computer Networks", "Software Engineering", 
           "Web Engineering", "Mobile Application Development", "Artificial Intelligence", "Machine Learning", "Data Science", "Internet of Things", "Cloud Computing", 
           "Cyber Security", "Network Security", "Discrete Mathematics", "Linear Algebra", "Calculus", "Applied Physics", "Computer Assembly", "Psychology","OOP"]
Teachers = ["Teacher 1", "Teacher 2", "Teacher 3", "Teacher 4", "Teacher 5", "Teacher 6", "Teacher 7","Teacher 8","Teacher 9","Teacher 10","Teacher 11","Teacher 12"]
Sections = ["Section 1", "Section 2", "Section 3","Section 4","Section 5","Section 6","Section 7","Section 8","Section 9","Section 10","Section 11","Section 12"]

Floors = 1
Classrooms = 4
Labs = 3

generations=[]

fitness = []
Classes = []
encodedGen=[]

TimeSlots = ['08:30', '10:05', '11:40', '01:05', '02:40', '04:05']


In [4]:
def create_classes():
    
    for i in range(Floors):
        temp = []
        for j in range(Classrooms):
            temp.append(f"Floor {i+1} Room {j+1} Capacity {60}")
        for j in range(Labs):
            temp.append(f"Floor {i+1} Lab {j+1} Capacity {120}")
        Classes.append(temp)

    # randomly making the capacity of 5 rooms 120
    for i in range(Floors):
        ran=random.randint(0, Classrooms-1)
        Classes[i][ran] = f"Floor {i+1} Room {ran+1} Capacity {120}"


In [5]:
# Example usage:
# Creating a random timetable Each day has 7 time slots, each time slot has classes in every classroom and lab of every floor has a random course, teacher, and section
create_classes()

# Have to chnage the days for only moday tuesday and friday
def TimeTableGenerator():
    TimeTable = {}
    for time_slot in TimeSlots:
        TimeTable[(time_slot)] = []
        for room in Classes[0]:
            course = random.choice(Courses)
            day = random.choice(Days) 
            section=random.choice(Sections)
            teacher = random.choice(Teachers)
            TimeTable[(time_slot)].append({
                    "day":day,
                    "section": section,
                    "course": course,
                    "room": room,
                    "teacher": teacher,
                    "type": "Lab" if "Lab" in room else "Theory"})
    return TimeTable



In [6]:

# generating chromosones in the form of a list with format
# Course, Theory/Lab, Section, Section-Strength, Professor, First-lecture-day,First- lecture-timeslot, First-lecture-room, First-lecture-room-size, Second-lecture-day, Second-lecture-timeslot, Second-lecture-room, Second-lecture-room-size
# for each course the lecture should be on different days
# the room size for alb is 120 vice versa 
# changing the time slot for lab to be consective
def generate_chromosomes(TimeTable):
    chromosomes = []
    encoded=[]
    for key, value in TimeTable.items():
        for val in value:
            if val["type"] == "Theory":
                valday2 = Days[(Days.index(val["day"]) + 2) % len(Days)] if val["day"] != "Friday" else random.choice(Days)
                room_size = 60
                timeslot1 = key
                timeslot2 = key
            else:
                valday2=val["day"]
                room_size = 120
                index = TimeSlots.index(key)
                if index == len(TimeSlots) - 1:
                    timeslot1= TimeSlots[-2]
                    timeslot2 = key
                else:
                    next_index = (index + 1) % len(TimeSlots)
                    timeslot1 = TimeSlots[index]
                    timeslot2 = TimeSlots[next_index]

            chromosomes.append([val["course"], val["type"], val["section"], 30, val["teacher"], val["day"], timeslot1, val["room"], room_size, valday2, timeslot2, val["room"], room_size])
    encode_data(chromosomes)
    return chromosomes

#Encoding the data in th ebinary encoding techniques

def encode_data(chromosomes):
    encoded = []

    for i in chromosomes: 
        hash_str = ''
        for j in i:
            if isinstance(j, str):
                for char in j:
                    hash_str += bin(ord(char))[2:].zfill(8)
                    hash_str += '|'
                hash_str += ':'
            elif isinstance(j, int):
                hash_str += bin(j)[2:].zfill(32)  
                hash_str += ';'
        hash_str += '!'
        encoded.append(hash_str)
    encodedGen.append(encoded)
    return encoded


In [7]:
def Fitness(chromosomes):
    # Initialize fitness score
    fitness = 0
  
    # Check if any teacher has more than 3 courses in a day
    for teacher in Teachers:
        courses = []
        for chromosome in chromosomes:
            if chromosome[4] == teacher:
                courses.append(chromosome[0])
        if len(set(courses)) > 3:
            fitness += 1

    # Check if any section has more than 5 courses in a day
    for section in Sections:
        courses = []
        for chromosome in chromosomes:
            if chromosome[2] == section:
                courses.append(chromosome[0])
        if len(set(courses)) > 5:
            fitness += 1

    # Constraints related to rooms, professors, sections, and courses
    for i in range(len(chromosomes)):
        for j in range(i + 1, len(chromosomes)):
            # A room cannot be assigned for two different sections at the same time.
            if len(chromosomes[i]) > 7 and len(chromosomes[j]) > 7:
                if chromosomes[i][7] == chromosomes[j][7]:
                    fitness += 1
            # A classroom should be big enough to accommodate the section.
            if chromosomes[i][8] < chromosomes[i][3]:
                fitness += 1
            # A professor should not be assigned two different lectures at the same time.
            if chromosomes[i][4] == chromosomes[j][4]:
                fitness += 1
            # The same section cannot be assigned to two different rooms at the same time.
            if chromosomes[i][2] == chromosomes[j][2]:
                fitness += 1

    # Constraints related to lectures per week for each course
    for section in Sections:
        for course in Courses:
            count = 0
            for chromosome in chromosomes:
                if chromosome[2] == section and chromosome[0] == course:
                    count += 1
            if count > 2:
                fitness += 1
            if count < 2:
                fitness += 1

    # Lab lectures should be conducted in two consecutive slots.
    for i in range(len(chromosomes)-1):
        if chromosomes[i][1] == "Lab" and chromosomes[i+1][1] == "Lab":
            if chromosomes[i][6] != chromosomes[i+1][6] or chromosomes[i][7] != chromosomes[i+1][7]:
                fitness += 1
    
    # Return negative of fitness score as it's a minimization problem
    return (-1) * fitness


In [8]:
def decode_data(encoded):
    decoded = []

    for encoded_str in encoded:
        decoded_chromosome = []
        parts = encoded_str.split('!')
        for part in parts:
            if part.endswith(':'):
                part = part[:-1]
            if ':' in part:
                chars = part.split(':')
                for char in chars:
                    if char:
                        if '|' in char:
                            char_binaries = char.split('|')
                            decoded_string = ''
                            for binary in char_binaries:
                                if binary:
                                    if ';' in binary:
                                        sub_binaries = binary.split(';')
                                        for sub_binary in sub_binaries:
                                            if sub_binary:
                                                decoded_string += chr(int(sub_binary, 2))
                                    else:
                                        decoded_string += chr(int(binary, 2))
                            decoded_chromosome.append(decoded_string)
            elif ';' in part:
                nums = part.split(';')
                for num in nums:
                    if num:
                        decoded_chromosome.append(int(num, 2))
        decoded.append(decoded_chromosome)
    return decoded



# Making Parent Selection,Two point Corossover and Mutation function for Genetic Algorithm to be run on the chromosomes
def ParentSelection(chromosomes):
    # generating 15 timetables and selecting the best 2
    TimeTables = []
    Fitnesses = []
    for i in range(5):
      TimeTable = TimeTableGenerator()
      TimeTables.append(TimeTable)
 
    for TimeTable in TimeTables:
        chromosomes = generate_chromosomes(TimeTable)
        generations.append(chromosomes)
        Fitnesses.append(Fitness(chromosomes))

    max_index = Fitnesses.index(max(Fitnesses))
    best1 = TimeTables[max_index]
    Fitnesses[max_index] = -1
    best2 = TimeTables[Fitnesses.index(max(Fitnesses))]
    Fitnesses[max_index] =best1
    return best1, best2

def TwoPointCrossover(parent1, parent2):
    # the parents are converted to chromosomes
    chromosomes1 = generate_chromosomes(parent1)
    chromosomes2 = generate_chromosomes(parent2)
    # 2 point crossover is performed
    crossover_point1 = random.randint(0, len(chromosomes1)-1)
    crossover_point2 = random.randint(0, len(chromosomes1)-1)
    if crossover_point1 > crossover_point2:
        crossover_point1, crossover_point2 = crossover_point2, crossover_point1
    child1 = chromosomes1[:crossover_point1] + chromosomes2[crossover_point1:crossover_point2] + chromosomes1[crossover_point2:]
    child2 = chromosomes2[:crossover_point1] + chromosomes1[crossover_point1:crossover_point2] + chromosomes2[crossover_point2:]
    return child1, child2

def Mutation(chromosome):
    # randomly changing the course, teacher, room, and section of a chromosome
    mutation_point = random.randint(0, len(chromosome)-1)

    chromosome[mutation_point][0] = random.choice(Courses)
    chromosome[mutation_point][2] = random.choice(Sections)
    chromosome[mutation_point][4] = random.choice(Teachers)

    return chromosome


In [9]:
def DisplayChromoes(chromosomes):
  for chromosome in chromosomes:
    # printing all the data in the chromosome exclusing the last index rather displaying that in binary form
    print(chromosome[:-1])

In [10]:
# contains the total generations produced
generations=[]
# Example mutation rate of 10%
mutation_rate = 0.1  
#Total number of generations
total_gen= 5

#generates the timetable
TimeTable = TimeTableGenerator()
# generate the complete chromosome by encoding it
chromosomes = generate_chromosomes(TimeTable)
#Dislays the chromose created at first
DisplayChromoes(chromosomes)


print("---------------------------------------------------------------->>>")
print("\n\n")
print("Runing for given number of generatrions")

# the main genetic algorithm starting........
for generation in range(total_gen):
    new_chromosomes = []
    for i in range(len(chromosomes)//2):
        parent1, parent2 = ParentSelection(chromosomes)
        child1, child2 = TwoPointCrossover(parent1, parent2)

        # Apply mutation with the given mutation rate
        if random.random() <= mutation_rate:
            child1 = Mutation(child1)
            child2 = Mutation(child2)

        new_chromosomes.append(child1)
        new_chromosomes.append(child2)

    chromosomes = new_chromosomes
    generations.extend(chromosomes)
    print("Generation:", generation, "Fitness:", Fitness(chromosomes))

# finding max fitness of the chromosomes and saving index and printing the timetable

for i in range(len(generations)):
    # print(chromosomes[i])
    fitness.append(Fitness(generations[i]))

index = fitness.index(max(fitness))


print("\n\n")
print("---------------------------------------------------------------->>>")
print("\n\n")
print(encodedGen[index])
print("\n\n")
print("---------------------------------------------------------------->>>")
print("\n\n")
print(decode_data(encodedGen[index]))
print("\n\n")
print("---------------------------------------------------------------->>>")
print("\n\n")
print("\n\nThe highest fitness individual is : \n")
for i in generations[index]:
    for j in i:
        print(i)
        print("\n")
print("\n\n")
print("---------------------------------------------------------------->>>")

['Software Engineering', 'Theory', 'Section 4', 30, 'Teacher 1', 'Wednesday', '08:30', 'Floor 1 Room 1 Capacity 120', 60, 'Friday', '08:30', 'Floor 1 Room 1 Capacity 120']
['Artificial Intelligence', 'Theory', 'Section 12', 30, 'Teacher 11', 'Monday', '08:30', 'Floor 1 Room 2 Capacity 60', 60, 'Wednesday', '08:30', 'Floor 1 Room 2 Capacity 60']
['Internet of Things', 'Theory', 'Section 5', 30, 'Teacher 7', 'Monday', '08:30', 'Floor 1 Room 3 Capacity 60', 60, 'Wednesday', '08:30', 'Floor 1 Room 3 Capacity 60']
['Machine Learning', 'Theory', 'Section 2', 30, 'Teacher 9', 'Wednesday', '08:30', 'Floor 1 Room 4 Capacity 60', 60, 'Friday', '08:30', 'Floor 1 Room 4 Capacity 60']
['Applied Physics', 'Lab', 'Section 12', 30, 'Teacher 9', 'Thursday', '08:30', 'Floor 1 Lab 1 Capacity 120', 120, 'Thursday', '10:05', 'Floor 1 Lab 1 Capacity 120']
['Applied Physics', 'Lab', 'Section 9', 30, 'Teacher 3', 'Friday', '08:30', 'Floor 1 Lab 2 Capacity 120', 120, 'Friday', '10:05', 'Floor 1 Lab 2 Capacity 